<a href="https://colab.research.google.com/github/SamLizotte/Machine-Learning-to-Detect-Alzheimers-Disease/blob/main/CNN_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget ivillar.github.io/datasetsforstudents/mridata.zip

--2023-11-07 20:34:38--  http://ivillar.github.io/datasetsforstudents/mridata.zip
Resolving ivillar.github.io (ivillar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to ivillar.github.io (ivillar.github.io)|185.199.110.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ivillar.github.io/datasetsforstudents/mridata.zip [following]
--2023-11-07 20:34:38--  https://ivillar.github.io/datasetsforstudents/mridata.zip
Connecting to ivillar.github.io (ivillar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29365000 (28M) [application/zip]
Saving to: ‘mridata.zip’

mridata.zip         100%[===================>]  28.00M  --.-KB/s    in 0.1s    

2023-11-07 20:34:38 (194 MB/s) - ‘mridata.zip’ saved [29365000/29365000]



In [ ]:
!unzip mridata.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Non_Demented/non_1395.jpg  
  inflating: Dataset/Non_Demented/non_1396.jpg  
  inflating: Dataset/Non_Demented/non_1397.jpg  
  inflating: Dataset/Non_Demented/non_1398.jpg  
  inflating: Dataset/Non_Demented/non_1399.jpg  
  inflating: Dataset/Non_Demented/non_14.jpg  
  inflating: Dataset/Non_Demented/non_140.jpg  
  inflating: Dataset/Non_Demented/non_1400.jpg  
  inflating: Dataset/Non_Demented/non_1401.jpg  
  inflating: Dataset/Non_Demented/non_1402.jpg  
  inflating: Dataset/Non_Demented/non_1403.jpg  
  inflating: Dataset/Non_Demented/non_1404.jpg  
  inflating: Dataset/Non_Demented/non_1405.jpg  
  inflating: Dataset/Non_Demented/non_1406.jpg  
  inflating: Dataset/Non_Demented/non_1407.jpg  
  inflating: Dataset/Non_Demented/non_1408.jpg  
  inflating: Dataset/Non_Demented/non_1409.jpg  
  inflating: Dataset/Non_Demented/non_141.jpg  
  inflating: Dataset/Non_Demented/non_1410.jpg  
  inflating: Dataset/N

In [ ]:
import glob
import cv2
import numpy as np
import keras
from keras import layers
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras import activations
from keras import backend as K

In [ ]:
verymild_images = []
files = glob.glob("Dataset/Very_Mild_Demented/*.jpg")
# non demented = 0; very mild demented = 1; mild demented = 2; moderate demented = 3
for filename in files:
  image = cv2.imread(filename)
  verymild_images.append(image)

verymild_array = np.array(verymild_images)

y_verymild = np.ones(verymild_array.shape[0])

mild_images = []
mildfile = glob.glob("Dataset/Mild_Demented/*.jpg")

for filename in mildfile:
  image = cv2.imread(filename)
  mild_images.append(image)

mild_array = np.array(mild_images)

y_mild = np.ones(mild_array.shape[0]) * 2

moderate_images = []
mod_files = glob.glob("Dataset/Moderate_Demented/*.jpg")

for filename in mod_files:
  image = cv2.imread(filename)
  moderate_images.append(image)

moderate_array = np.array(moderate_images)

y_moderate = np.ones(moderate_array.shape[0]) * 3

non_demented_images = []
nondem_files = glob.glob("Dataset/Non_Demented/*.jpg")

for filename in nondem_files:
  image = cv2.imread(filename)
  non_demented_images.append(image)

nondem_array = np.array(non_demented_images)

y_nondem = np.ones(nondem_array.shape[0]) * 0

In [ ]:
mri_images = np.concatenate((verymild_array,mild_array,moderate_array,nondem_array), casting="same_kind")

y_mri_images = np.concatenate((y_verymild, y_mild, y_moderate, y_nondem))

def normalize_image(image):


  return (image - np.min(image)) / (np.max(image) - np.min(image))

norm_mri = []
for img in mri_images:
  norm_mri.append(normalize_image(img))

norm_mri = np.array(norm_mri)
norm_mri.shape

(6400, 128, 128, 3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(norm_mri, y_mri_images, random_state=None, shuffle=True, stratify=None)

In [ ]:
#RUN THIS
#import VGG19
from keras.applications import VGG19

img_rows, img_cols = 128, 128
cnn_vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(img_rows, img_cols, 3))


80134624/80134624 [==============================] - 3s 0us/step


In [ ]:
#RUN THIS
#vgg19 fine tuning
input_shape = (img_rows, img_cols, 3)

cnn = Sequential()
for layer in cnn_vgg19.layers:
    cnn.add(layer)
cnn.add(BatchNormalization(synchronized=True))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.25)), cnn.add(Flatten())
cnn.add(Dense(128, activation = 'relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(64, activation = 'relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation = 'softmax'))

cnn.compile(loss = keras.losses.categorical_crossentropy,
   optimizer = keras.optimizers.Adadelta(), metrics = ['accuracy'])

In [ ]:
#RUN THIS
img_rows, img_cols = 128, 128

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
input_shape = (img_rows, img_cols, 3)


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
#RUN THIS
#trainability
num_layers = len(cnn.layers)
for x in range(0,num_layers):
    print(cnn.layers[x])
    print(cnn.layers[x].trainable)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
cnn = Sequential()
cnn.add(Conv2D(128, kernel_size=(5, 5), input_shape = input_shape))
cnn.add(BatchNormalization(synchronized=True))
cnn.add(layers.Activation(activations.relu))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(256, kernel_size=(5, 5), input_shape = input_shape))
cnn.add(BatchNormalization(synchronized=True))
cnn.add(layers.Activation(activations.relu))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(512, kernel_size=(5, 5), input_shape = input_shape))
cnn.add(BatchNormalization(synchronized=True))
cnn.add(layers.Activation(activations.relu))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(1028, kernel_size=(5, 5), input_shape = input_shape))
cnn.add(BatchNormalization(synchronized=True))
cnn.add(layers.Activation(activations.relu))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.25)), cnn.add(Flatten())
cnn.add(Dense(128, activation = 'relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(64, activation = 'relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation = 'softmax'))

cnn.compile(loss = keras.losses.categorical_crossentropy,
   optimizer = keras.optimizers.Adadelta(), metrics = ['accuracy'])

In [ ]:
#RUN THIS

cnn.fit(X_train,y_train,
         batch_size = 128,
         epochs = 250,
         verbose = 1,
         validation_data = (X_test, y_test))



Epoch 1/250
38/38 [==============================] - 68s 1s/step - loss: 3.7792 - accuracy: 0.1644 - val_loss: 2.2339 - val_accuracy: 0.4988
Epoch 2/250
38/38 [==============================] - 28s 733ms/step - loss: 2.4803 - accuracy: 0.2492 - val_loss: 2.1693 - val_accuracy: 0.4988
Epoch 3/250
38/38 [==============================] - 28s 742ms/step - loss: 2.1766 - accuracy: 0.3077 - val_loss: 2.1164 - val_accuracy: 0.4988
Epoch 4/250
38/38 [==============================] - 28s 731ms/step - loss: 2.0614 - accuracy: 0.3265 - val_loss: 2.0727 - val_accuracy: 0.4988
Epoch 5/250
38/38 [==============================] - 28s 726ms/step - loss: 1.9537 - accuracy: 0.3454 - val_loss: 2.0294 - val_accuracy: 0.4988
Epoch 6/250
38/38 [==============================] - 28s 740ms/step - loss: 1.8692 - accuracy: 0.3596 - val_loss: 1.9983 - val_accuracy: 0.4988
Epoch 7/250
38/38 [==============================] - 28s 742ms/step - loss: 1.8279 - accuracy: 0.3758 - val_loss: 1.9666 - val_accuracy: 0.

In [ ]:
#RUN THIS
cnn.predict(X_test)

50/50 [==============================] - 5s 44ms/step


array([[7.5383618e-02, 9.0849954e-01, 4.0642158e-03, ..., 5.9043459e-04,
        2.8412617e-04, 1.1170649e-03],
       [5.9614159e-02, 9.0725386e-01, 8.3166128e-03, ..., 7.2470988e-04,
        5.2220945e-04, 3.5777083e-03],
       [6.3146573e-01, 2.0902063e-01, 6.8279728e-02, ..., 4.1844407e-03,
        1.3010106e-02, 1.5311255e-02],
       ...,
       [9.8836845e-01, 9.6371071e-03, 2.2564073e-04, ..., 4.3519012e-05,
        4.0537496e-05, 3.3172261e-04],
       [9.9993396e-01, 4.1902535e-05, 6.9440836e-07, ..., 3.4201658e-08,
        3.2801810e-07, 6.1506189e-06],
       [5.1767819e-02, 8.3375192e-01, 3.7607621e-02, ..., 3.4389314e-03,
        2.2519815e-03, 1.1020360e-02]], dtype=float32)

In [ ]:
#test size 0.7 w/o batch normalization - 49% accuracy
#test size 0.5 w/o batch normalization - 50% accuracy
#100 epochs with 64/128/256 conv2d neurons -> 58% accuracy
#150 epochs with 64/128/256/512 conv2d neurons -> 72% accuracy
#200 epochs with 64/128/256/512 conv2d neurons -> 77% accuracy
#200 epochs with 128/256/512/1028 conv2d neurons -> 91% accuracy
#250 epochs with 128/256/512/1028 conv2d neurons -> 93% accuracy
#300 epochs with 128/256/512/1028 conv2d neurons -> 93% accuracy

#VGG19/150 epochs -> 50% accuracy

##250 epochs with 128/256/512/1028 conv2d neurons -> 93% accuracy a
score = cnn.evaluate(X_test, y_test)

print("Loss:", score[0])
print("Accuracy:", score[1])

50/50 [==============================] - 3s 50ms/step - loss: 0.2979 - accuracy: 0.9350
Loss: 0.29791802167892456
Accuracy: 0.9350000023841858
